### Load dependencies

In [158]:
import os
import shutil
import time
import pickle
import pandas as pd
import matplotlib.pyplot as pyplot
import librosa
import librosa.display
import gc
import numpy as np
import urllib

def download_rec(uri, rec_dir, filename):
    ### Downloads the uri to the rec_dir/filename
    if not os.path.exists(rec_dir+'/'+uri.replace('/','_')):
        try:
            urllib.request.urlretrieve (uri, rec_dir+'/'+filename)
        except Exception as e:
            print(e)
            print('Error downloading '+uri)
        return
    

### Specify data paths:

1. train_set_dir - Folder where training data will be stored
2. recording_dir - Folder where audio recordings will be stored
3. sound_annotation_file - File storing template matching validation metadata
4. (Optional) sampling_rate - rate to resample training data recordings to
    

In [159]:
data_dir = '/home/jupyter/arbimon2-cnn/data/' #Added by andrew
train_set_dir = '/home/jupyter/arbimon2-cnn/data/train_tp/' # Folder where training data will be stored - true positives
train_dir_fp = '/home/jupyter/arbimon2-cnn/data/train_fp/' # directory with examples of false-positive spectrograms of each class
recording_dir = '/home/jupyter/arbimon2-cnn/data/recordings/' # Folder holding recordings
# sound_annotation_files = ['./example_annotations.csv'] # pattern matching results from Arbimon
sound_annotation_files = ['./Demo_MCC.csv'] # pattern matching results from Arbimon
# File storing ROIs of detected sounds (animal calls) 
#     Required columns:
#          species
#          x1 (start time of sound)
#          x2 (end time of sound)
#          url (recording file path)

sampling_rate = 48000 # training data recording sample rate

### Run remaining cells to generate training data

In [160]:
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
if not os.path.exists(recording_dir):
    os.mkdir(recording_dir)
if not os.path.exists(train_set_dir):
    os.mkdir(train_set_dir)
if not os.path.exists(train_dir_fp):
    os.mkdir(train_dir_fp)    

In [161]:
if len(sound_annotation_files)==1:
    all_rois= pd.read_csv(sound_annotation_files[0])
    is_validated = all_rois['validated']!='(not validated)'
    all_rois = all_rois[is_validated] # only the validated recordings
elif len(sound_annotation_files)==0:
    print('Must provide an annotation file')
elif len(sound_annotation_files)>1:
    all_rois = pd.read_csv(sound_annotation_files[0])
    for i in sound_annotation_files[1:]:
        tmp = pd.read_csv(sound_annotation_files[i])
        all_rois = pd.concat([all_rois,tmp])
all_rois.head()

,id,recording,site,year,month,day,hour,minute,species,songtype,x1,x2,y1,y2,frequency,validated,url,score,site_id
9,96849321,5ADE0894.flac,BCI03,2018,4,23,16,23,Contopus virens,Common Song,9.253333,10.346667,2718.75,4781.25,24000,present,https://arbimon.rfcx.org/api/project/bci-panam...,0.559809,3094
10,96849322,5ADE0894.flac,BCI03,2018,4,23,16,23,Contopus virens,Common Song,51.381333,52.474667,2718.75,4781.25,24000,present,https://arbimon.rfcx.org/api/project/bci-panam...,0.559508,3094
15,96849327,5AE95269.flac,BCI04_2_dorms,2018,5,2,5,53,Contopus virens,Common Song,40.746667,41.840000,2718.75,4781.25,24000,not present,https://arbimon.rfcx.org/api/project/bci-panam...,0.248756,3191
19,96849331,5AD4E121.flac,BCI17,2018,4,16,17,45,Contopus virens,Common Song,31.578667,32.672000,2718.75,4781.25,24000,not present,https://arbimon.rfcx.org/api/project/bci-panam...,0.200485,3146
22,96849334,5AD82F28.flac,BCI02,2018,4,19,5,54,Contopus virens,Common Song,11.328000,12.421333,2718.75,4781.25,24000,not present,https://arbimon.rfcx.org/api/project/bci-panam...,0.204940,3127


In [162]:
print('Number of ROIs for each species\n')
for i in list(set(all_rois.species)):
    print(str(i)+'\t\t'+str(len(all_rois[all_rois.species==i])))

Number of ROIs for each species

Contopus virens		133


In [163]:
# Save the audio recording and the spectrogram to the folders created above
window_length = 2 # sample time-window length in seconds
t0 = time.time()
for url in list(set(all_rois.url)): # get the unique urls
    print(url)
    rec_loaded = False
    # get all of the ROIs for the current url
    rois = all_rois[all_rois.url==url] 
    # loop over spectrogram ROIs
    for c in range(len(rois)): 
        try:
            print(rois.iloc[c].species + ' ' + rois.iloc[c].validated + ' ' + str(rois.iloc[c].score))
            # get the start and end times of the ROIs
            sound_start, sound_end = [rois.iloc[c].x1, rois.iloc[c].x2]
            # get the species name
            species = rois.iloc[c].species.replace(' ','_')
            # get the audio filename
            audio_filename = rois.iloc[0].recording
            # create the species name folder if it doesnt already exist
            if not os.path.exists(train_set_dir + '/' + str(species)):
                os.mkdir(train_set_dir + '/' + str(species))
            if not os.path.exists(train_dir_fp + '/' + str(species)):
                os.mkdir(train_dir_fp + '/' + str(species))
            if rois.iloc[c]['validated']=='present':
                folder = train_set_dir
            else:
                folder = train_dir_fp
            # get the shift in time to be able to standardise all samples to 2 seconds long, e.g. if the sound is 1.4s long the shift will be 0.3s and so there will be 0.3s before and 0.3s after to make the sample 2s long
            shft = ((sound_end - sound_start) - window_length)/2
            start_sample = round(sampling_rate * (sound_start + shft)) # e.g. 24s in will be (48000*24) = 1152000
            # ensure the start time is not <0
            start_sample = max(start_sample, 0)
            # append the start and end times of the sample into the filename of the spectrogram
            # e.g. project_1_site_506_2014_4_sabana_seca1-2014-04-04_03-20_22.3-24.3
            #                      is from 04/04/2014 at 03:20 and between 22.3s to 24.3s
            # get the folder where the spectrogram will be created
            filename = rois.iloc[0].recording.split('.')[0] + '_' + str(round(start_sample/sampling_rate,2)) + '-' + str(round((start_sample/sampling_rate) + window_length,2)) + '.png'
            # if the spectrogram has not been created - create it and write it to the train_set_dir folder
            print(folder + str(species) + '/' + filename)
            if not os.path.exists(folder + str(species) + '/' + filename):
                if not rec_loaded: # has the recording been downloaded
                    if not os.path.exists(recording_dir + '/' + audio_filename): 
                        # download the recording
                        download_rec(url, recording_dir, audio_filename)
                    try:
                        # load the audio file
                        audio_data, sampling_rate = librosa.load(recording_dir+audio_filename, sr=sampling_rate)
                        rec_loaded = True
                    except Exception as e:
                        print(e)
                        continue
                # create the spectrogram
                S = librosa.feature.melspectrogram(y = audio_data[int(start_sample): int(start_sample+round(sampling_rate*window_length))], 
                                               sr = sampling_rate,
                                               n_fft=2048, 
                                               hop_length=512, 
                                               win_length=1024)
                dpi=100
                fig = pyplot.figure(num=None, figsize=(300/dpi, 300/dpi), dpi=dpi)
                pyplot.subplot(222)
                ax = pyplot.axes()
                ax.set_axis_off()
                librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
                # save the spectrogram to file
                pyplot.savefig(folder+str(species.replace(' ','_'))+'/'+filename, bbox_inches='tight', transparent=True, pad_inches=0.0)
                pyplot.close()
                
        except Exception as e:
            print(e)
            continue
        
    rec_loaded = False    

https://arbimon.rfcx.org/api/project/bci-panama-2018/recordings/download/3355783
Contopus virens present 0.4759610536910118
/home/jupyter/arbimon2-cnn/data/train_tp/Contopus_virens/5ADDFD18_36.83-38.83.png
https://arbimon.rfcx.org/api/project/bci-panama-2018/recordings/download/3473710
Contopus virens not present 0.2487563816946783
/home/jupyter/arbimon2-cnn/data/train_fp/Contopus_virens/5AE95269_40.29-42.29.png
https://arbimon.rfcx.org/api/project/bci-panama-2018/recordings/download/3612467
Contopus virens not present 0.3080719745762741
/home/jupyter/arbimon2-cnn/data/train_fp/Contopus_virens/5AE4131B_32.77-34.77.png
https://arbimon.rfcx.org/api/project/bci-panama-2018/recordings/download/3380982
Contopus virens present 0.5425935560452593
/home/jupyter/arbimon2-cnn/data/train_tp/Contopus_virens/5ADF16D6_18.76-20.76.png
https://arbimon.rfcx.org/api/project/bci-panama-2018/recordings/download/3413919
Contopus virens present 0.592735169472384
/home/jupyter/arbimon2-cnn/data/train_tp/Cont